In [ ]:
%matplotlib inline

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz

dataset = foz.load_zoo_dataset(
    "open-images-v6",
    split="train",
    classes=["Person", "Horse"],
    label_types=["detections"],
    max_samples=2000,
    shuffle=True,
)

In [ ]:
# Optionally view the dataset
session = fo.launch_app(dataset)

In [ ]:
from fiftyone import ViewField as F, DatasetView

valid_labels = ["Person", "Horse"]
valid_labels_view: DatasetView = dataset.filter_labels(
    "detections", F("label").is_in(valid_labels)
)

print(len(valid_labels_view))

In [ ]:
# The directory to which to write the exported dataset
export_dir = "dataset"

# The name of the sample field containing the label that you wish to export
# Used when exporting labeled datasets (e.g., classification or detection)
label_field = "detections"  # for example

# The type of dataset to export
# Any subclass of `fiftyone.types.Dataset` is supported
dataset_type = fo.types.VOCDetectionDataset

# Export the dataset
valid_labels_view.export(
    export_dir=export_dir,
    dataset_type=dataset_type,
    label_field=label_field,
)

Walk over the label XML and add some information necessary for Tensorflow's Model Maker, writing out the transformation to a `dataset/tf_labels` folder.

In [ ]:
import os
import xml.etree.ElementTree as ET

mappings = {"Person": 1, "Horse": 2}
labels_dir = "dataset/labels"
tf_labels_dir = "dataset/tf_labels"

os.makedirs(tf_labels_dir, exist_ok=True)

for label_file_name in os.listdir(labels_dir):
  path = f"{labels_dir}/{label_file_name}"
  tf_path = f"{tf_labels_dir}/{label_file_name}"
  doc = ET.parse(path)
  root = doc.getroot()

  for obj_el in root.iter("object"):
    ET.SubElement(obj_el, "pose")
    ET.SubElement(obj_el, "difficult").text = "0"

    truncated_el = obj_el.find("IsTruncated")
    truncated_flag = "0"
    if truncated_el is None: 
      truncated_flag = "0" 
    elif truncated_el.text == "True":
      truncated_flag = "1"
    ET.SubElement(obj_el, "truncated").text = truncated_flag

  doc.write(tf_path)
